# K-means кластеризація для сегментації користувачів, заснованої на RFM змінних

У цьому ДЗ вирішуватимемо задачу сегментації користувачів, виходячи з їх нещодавності та частоти покупок, а також сум витрат на них.

Імпорти розташуйте під цією клітиною.

In [197]:
import pandas as pd
from numpy import NaN

Будемо використовувати набір даних із наступного джерела: https://archive.ics.uci.edu/ml/datasets/online+retail

1.Завантажте набір даних із файлу. Виведіть перші 5 записів та розмір набору даних.

In [46]:
df = pd.read_excel('https://github.com/dmytrovelychko/rd_ds_python/raw/main/datasets/hw_6.xlsx')

In [47]:
print(df.shape)
df.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [19]:
df[df["StockCode"] == "85123A" ].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom


## Підготовка даних

2. Проведіть невеликий аналіз даних.
2.1.Виведіть результат методу describe() для числових колонок. Виведіть кількість унікальних значень у кожній категоріальній колонці.

In [18]:
df[["Quantity", "UnitPrice"]].describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


min і max мають колосальні значення, особливо для quantity

In [20]:
# Description = StockCode
df["StockCode"].value_counts()

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
21431        1
22275        1
17001        1
90187A       1
72759        1
Name: StockCode, Length: 4070, dtype: int64

In [21]:
df["Country"].value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

2.1.1.Як можна пояснити те, що у змінних Quantity і UnitPrice мінімальне значення негативне?

In [72]:
df_quantity_lt_0 = df[df["Quantity"] < 0 ]
df_quantity_lt_0.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


Quantity < 0, в інвойсах які описують відміну транзакції, згідно InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

Подивимось чи бувають інші випадки

In [78]:
df_quantity_lt_0_not_cancelled = df_quantity_lt_0[df["InvoiceNo"].apply(lambda x: str(x).startswith("C")) == False]
df_quantity_lt_0_not_cancelled.head()

<ipython-input-78-148810a93c5c>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_quantity_lt_0_not_cancelled = df_quantity_lt_0[df["InvoiceNo"].apply(lambda x: str(x).startswith("C")) == False]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,NaN,United Kingdom


як бачимо є інвойси не помічені як відміна але з відʼємною кількістю, перевіримо чи трапляются такі для конкретного користувача

In [87]:
df_quantity_lt_0_not_cancelled["CustomerID"].notna().value_counts()

False    1336
Name: CustomerID, dtype: int64

Отже всі інвойси які не позначені як відміна і мають відʼємне значення не привʼязані до користувача і будуть пізніше відсіяні із виборки, адже CustomerID в даних умовах не вийде запроксимувати (недостатньо додаткових даних)

In [89]:
df[df["UnitPrice"] < 0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [26]:
df[df["StockCode"] == "B"]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom



UnitPrice < 0, це наслідок перерахунку для зведення балансу

Заради цікавості оцінемо інвойси, коли UnitPrice == 0 або Quantity == 0.

In [97]:
df_with_cid = df[df["CustomerID"].notna()]
print(df_with_cid[df_with_cid["Quantity"] == 0].shape)
print(df_with_cid[df_with_cid["UnitPrice"] == 0].shape)

(0, 8)
(40, 8)


як бачимо кількість не значна і можна знехтувати, бо це стосується малої кількісті користувачів, а для RFM може вплинути тільки на Recency і Frequency показниках

2.1.2. Як ви вважаєте, чи варто залишати ці дані (з негативними значеннями Quantity і UnitPrice) у наборі даних, виходячи з задачі, яку ми вирішуємо? Аргументуйте свою відповідь.
Якщо ви вважаєте, що ці рядки треба видалити, видаліть їх.

In [58]:
customers_with_cancelletions = df[df["Quantity"] < 0].groupby("CustomerID")["CustomerID"].count()
customers_with_cancelletions.shape

(1589,)

In [42]:
(customers_with_cancelletions.shape[0]/df.groupby("CustomerID").count().shape[0])*100

36.34492223238792

36.35% відсотків користувачів робили відміни (можливо це мається на увазі і повернення товару) з 01/12/2010 по 09/12/2011. Це значна кількість і може бути дослідженна як окрема катеогрія користувачів. У рамках RFM аналізу, їх видалення може дати біас по реальним витратам конкретного користувача. Подивимось як на конкретному прикладі

In [65]:
customers_with_cancelletions.sort_values()

CustomerID
12346.0      1
15046.0      1
15048.0      1
15079.0      1
15091.0      1
          ... 
12607.0    101
15311.0    112
17511.0    113
17841.0    136
14911.0    226
Name: CustomerID, Length: 1589, dtype: int64

In [64]:
df[df["CustomerID"] == 12346]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom


Як бачимо якщо видалити запис відміни для користовуча 12346, ми отримаємо його витрати у вигляді Quantity*UnitPrice=77183,6 що в корні не вірно, тому потрібно або видаляти оба записи (що буде важко ідентифікувати в пари) або враховувати це при подальших обчисленнях

UnitPrice < 0, всього 3 записи, вони не привʼязані до користувача, я перевірив що вони і не привʼязані до інвойсів з іншими видами продуктів, тому в рамках сегментації користувачів їх треба видалити

In [98]:
df = df[df["UnitPrice"] >= 0]

2.2.Виведіть відсоток відсутніх значень у кожній із колонок.

In [101]:
(df.isna().sum() / df.shape[0]) * 100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268312
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926417
Country         0.000000
dtype: float64

Descripton це додаткова інформація до StockCode, відсутні CustomerID не лікуються, видалимо тільки відсутні CustomerID

2.4. Видаліть рядки з відсутніми значеннями.

In [106]:
df = df[df["CustomerID"].notna()]

2.5. Виведіть значення змінної Country, відсортувавши їх за відсотком присутності в даних.
Тобто вгорі має бути значення, яке найчастіше є у даних.

In [111]:
df["Country"].value_counts().sort_values(ascending=False)

United Kingdom          361878
Germany                   9495
France                    8491
EIRE                      7485
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1480
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     250
Unspecified                244
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45
Lithuani

2.6.У наборі даних є значення змінної Country, яке зустрічається найчастіше.
Відфільтруйте дані лише до тих записів, які мають значення змінної Country. Це спростить нам задачу, до того ж для інших значень Country обмаль даних.

In [113]:
df = df[df["Country"] == "United Kingdom"]
df.shape

(361878, 8)

2.7. Створіть стовпець Date із значень у стовпці InvoiceDate. Стовпець Date повинен містити дату у форматі рік-місяць-день, наприклад "2021-10-24".

In [136]:
df["InvoiceDate"] = df["InvoiceDate"].apply(lambda x: x.date())

In [135]:
df["InvoiceDate"].head()

0    2010-12-01
1    2010-12-01
2    2010-12-01
3    2010-12-01
4    2010-12-01
Name: InvoiceDate, dtype: object

Отже, на даний момент у нас є всі необхідні дані для вирішення задачі:
- Customer ID
- Unit Price
- Quantity
- Invoice Date

## RFM

Давність-частота-грошовий вираз (Recency-Frequency-Monetary)
    
    
RFM-аналіз дозволяє сегментувати клієнтів за частотою та вартістю покупок та визначати тих клієнтів, які витрачають найбільше грошей.

* Давність - скільки часу пройшло з того часу, як клієнт щось купив у нас.
* Частота - як часто клієнт купує у нас.
* Грошовий вираз - загальна вартість покупок, здійснених покупцем.

Логічно припустити з опису вище, що **дані мають бути згруповані за клієнтами**. Інакше немає сенсу виводити Recency та Частоту.

3.1.Визначте останню дату транзакції у наборі даних. Ця дата нам необхідна, щоб визначати, як давно користувач робив покупку. Але дані історичні і якщо ми візьмемо для розрахунку recency фактичну сьогоднішню дату, розподіл recency буде зрушений у бік кількох сотень днів.
Щоб розподіл recency розпочинався з нуля, за дату "сьогодні" ми візьмемо дату останньої транзакції в наборі даних. Запишіть знайдену дату змінної NOW.

In [138]:
NOW = df["InvoiceDate"].max()
NOW

datetime.date(2011, 12, 9)

### Recency

3.2.У змінну R_df запишіть recency dataframe, що складається з колонок CustomerID, Last_Purchase_Date та Recency,
де     
   - Last_Purchase_Date - це дата останньої покупки клієнта;
   - Recency - скільки днів минуло з дати останньої покупки клієнта до NOW.

In [160]:
R_df = df.groupby("CustomerID")["InvoiceDate"].max().to_frame("Last_Purchase_Date")
R_df["Recency"] = NOW - R_df["Last_Purchase_Date"]
R_df.head()

,Last_Purchase_Date,Recency
CustomerID,,
12346.0,2011-01-18,325 days
12747.0,2011-12-07,2 days
12748.0,2011-12-09,0 days
12749.0,2011-12-06,3 days
12820.0,2011-12-06,3 days


### Frequency - Monetary

3.3.Створіть Frequency - Monetary dataframe у змінній FM_df.  
Датафрейм повинен містити
- CustomerID - ідентифікатор клієнта
- Frequency - кількість покупок клієнта (кількість унікальних інвойсів)
- Monetary - скільки грошей у магазині витратив клієнт

In [161]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom


При підрахунку кількості унікальних інвойсів треба врахувати інвойси з відміною покупки, тому для кожного користувача відніму від загальної кількості унікальних інвойсів 2 рази кількість відмін. Виходжу із гіпотези що при підрахунку частоти покупок нас цікавить саме завершені покупки, також нехтую ситуаціями коли транзакція покупки НЕ попала в таймлайн датасету, а відміна попала

In [195]:
cid_invoice_raw_count = df.groupby("CustomerID")["InvoiceNo"].nunique()
cid_invoice_raw_count.head()

CustomerID
12346.0      2
12747.0     11
12748.0    224
12749.0      8
12820.0      4
Name: InvoiceNo, dtype: int64

In [194]:
cid_invoice_cancelled_count = df[["CustomerID", "InvoiceNo"]].copy()
cid_invoice_cancelled_count = cid_invoice_cancelled_count.set_index("CustomerID")
# will exclude not cancelled invoices from unique counts
cid_invoice_cancelled_count["InvoiceNo"] = cid_invoice_cancelled_count["InvoiceNo"].apply(lambda x: NaN if not str(x).startswith("C") else x)
cid_invoice_cancelled_count = cid_invoice_cancelled_count.groupby("CustomerID")["InvoiceNo"].nunique()

cid_invoice_cancelled_count.head()

CustomerID
12346.0     1
12747.0     0
12748.0    14
12749.0     3
12820.0     0
Name: InvoiceNo, dtype: int64

In [196]:
cid_invoice_raw_count - 2*cid_invoice_cancelled_count

CustomerID
12346.0      0
12747.0     11
12748.0    196
12749.0      2
12820.0      4
          ... 
18280.0      1
18281.0      1
18282.0      1
18283.0     16
18287.0      3
Name: InvoiceNo, Length: 3950, dtype: int64

### RFM
3.4.Створіть RFM_df, який є об'єднанням R_df та FM_df за ідентифікатором клієнта.

In [ ]:
# ваш код тут

## K-means для RFM сегментації

K-means дає кращий результат за наступних умов:
- Розподіл даних не спотворений (тобто немає довгих хвостів)
- Дані стандартизовані (тобто середнє значення 0 та стандартне відхилення 1).

4.1.Побудуйте діаграми розподілу даних (наприклад, sns.distplot) для колонок Recency, Frequency і Monetary RFM_df.
Виведіть коефіцієнт асиметрії кожної з колонок.

In [ ]:
# ваш код тут

Дані сильно перекручені, тому ми виконаємо над ними log перетворення, щоб зменшити асиметрію кожної змінної.

4.2.Над кожною зі змінних 'Recency', 'Frequency', 'Monetary' виконайте перетворення log(x+1).   
Додаємо константу, оскільки log перетворення вимагає, щоб усі значення були позитивними.
Відобразіть розподіл змінних після перетворення.

In [ ]:
# ваш код тут

4.3.Бачимо, що в нас все одно залишилися хвости. Приберемо їх з використанням техніки **Winsorizing**.    

Прочитайте про неї тут: https://en.wikipedia.org/wiki/Winsorizing.    

Для використання техніки будемо використовувати бібліотеку [feature_engine](https://feature-engine.readthedocs.io/en/1.1.x/).    
Встановіть її.


Нам потрібний об'єкт Winsorizer. Імпортуйте його за допомогою
`from feature_engine.outliers import Winsorizer`.

У змінній windsoriser створіть екземпляр класу Winsorizer зі значеннями змінних 'tail='both'', 'fold=2', а в 'variables' передайте назви змінних, які ми хочемо перетворити.
Викличте метод fit().

In [ ]:
# ваш код тут
windsoriser = Winsorizer(...)

Викликаємо трансформацію:

In [ ]:
... = windsoriser.transform(...)

4.4. Стандартизуйте наші RFM змінні так, щоб середнє було 0, а стандартне відхилення дорівнювало 1. Тут не треба ділити на трейн і тест, можемо працювати разом з повним набором даних.

In [ ]:
# ваш код тут

4.5. Знайдіть оптимальну кількість кластерів за допомогою Elbow Method.
Запустіть KMeans кластеризацію підготовлених стандартизованих RFM даних з параметром K від 1 до 10 включно.
Виконати це можна у циклі.
Відобразіть графічно отримані результати про значення сум внутрішньокластерних відстаней (у sklearn його називають inertia) для різних значень K.

In [ ]:
# ваш код тут

Значення кластера, при якому зменшення значення inertia стає постійним, можна вибрати як правильне значення кластера для наших даних.
Спробуємо використовувати значення К від 3 до 5.

4.6.Створіть функцію, яка прийматиме набір даних для навчання (стандартизовані дані), значення К та набір даних з оригінальними (до стандартизації) значеннями (назвем його original_df_rfm).

Усередині функції необхідно
1. Навчити на стандартизованих даних Kmeans з кількістю кластерів К, що є аргументом функції.
2. Отримати ідентифікатори кластерів (лейбли).
3. Створити новий датафрейм як копію original_df_rfm.
4. До набору даних, отриманого в п.3, додайте колонку зі значенням кластера для даного рядка.
5. Виконайте зменшення розмірності за допомогою TSNE (можете спробувати замість TSNE використовувати PCA, і візуально порівняти результати).
6. Відобразити результати кластеризації.

Пункти 5 та 6 вже реалізовані за вас.
Запустіть функцію для значень K = {3,4,5}.
**Наша мета** - за допомогою візуалізацій зрозуміти, яка кількість кластерів оптимальна.

In [ ]:
from sklearn.manifold import TSNE

def fit_visualize_kmeans(..., clusters_number, original_df_rfm):

    ...
    cluster_labels = ...

    # Копия original_df_rfm с доп колонкой, содержащей лейблы кластеров
    df_new = ...

    # Инициируем TSNE
    model = TSNE(random_state=1)
    transformed = model.fit_transform(df_new)

    # Визуализируем результаты TSNE трансформации
    plt.title('Flattened Graph of {} Clusters'.format(clusters_number))
    params = dict(
        x=transformed[:,0],
        y=transformed[:,1],
        hue=cluster_labels,
        style=cluster_labels,
        palette="Set1"
    )
    sns.scatterplot(**params)
    return df_new

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(3, 1, 1)
df_rfm_k3 = fit_visualize_kmeans(...)

plt.subplot(3, 1, 2)
df_rfm_k4 = fit_visualize_kmeans(...)

plt.subplot(3, 1, 3)
df_rfm_k5 = fit_visualize_kmeans(...)

plt.tight_layout()


Проаналізуйте отримані графіки. При якій кількості кластерів на вашій основі з візуалізацій найкраще поділяються дані на кластери?

4.7.Збудуємо додатково [Snake plots](https://www.marketingprofs.com/tutorials/snakeplot.asp), які часто використовують у маркетингових аналізах.

Прочитайте про цей графік та давайте напишемо функцію для її реалізації.
У функції має відбуватися таке:
1. Створюємо датафрейм normalised_df_rfm, який містить оригінальні CustomerID, стандартизовані значення Recency, Frequency, Monetary та номер кластера.
2. Візуалізуємо датафрейм. Ця частина вже реалізована.  

Визначте відповідні аргументи функції самостійно. Вона має запускатися для різних результатів кластеризації.
Функція snake_plot може приймати декілька параметрів.

In [ ]:
def snake_plot(...):
    normalised_df_rfm = ...

    # "Розправляємо " дані в довгий формат
    df_melt = pd.melt(normalised_df_rfm.reset_index(),
                        id_vars=['CustomerID', 'Cluster'],
                        value_vars=['Recency', 'Frequency', 'Monetary'],
                        var_name='Metric',
                        value_name='Value')
    plt.xlabel('Metric')
    plt.ylabel('Value')
    sns.pointplot(data=df_melt, x='Metric', y='Value', hue='Cluster')

In [ ]:
plt.figure(figsize=(9, 9))

plt.subplot(3, 1, 1)
plt.title('Snake Plot of K-Means = 3')
snake_plot(...)

plt.subplot(3, 1, 2)
plt.title('Snake Plot of K-Means = 4')
snake_plot(...)

plt.subplot(3, 1, 3)
plt.title('Snake Plot of K-Means = 5')
snake_plot(...)

plt.tight_layout()

Зі згладжених графіків і графіків-змій видно, що значення кластера, що дорівнює 4, добре сегментує наших клієнтів.
Ми також могли б піти на більшу кількість кластерів, що повністю залежить від того, як компанія хоче сегментувати своїх клієнтів. Але зазвичай 4 кластери – це зручно.

4.8.Інтерпретуємо результати кластеризації.
З результатів кластеризації з кількістю кластерів 4 (і оригінальними значеннями RFM змінних) сформуйте агрегат , який містить середнє значення по кожній із трьох колонок та кількість клієнтів у кластері.
Виведіть цей агрегат.

In [ ]:
# ваш код тут

Зробіть висновки про отримані результати. Як би ви назвали кожний кластер? Які з них містять користувачів, що платять, а які - тих, хто потенційно піде у відтік?

    ваш відповідь тут